In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("twitter_transformation").getOrCreate()

23/09/20 11:26:07 WARN Utils: Your hostname, bebeto-HP-EliteDesk-800-G4-DM-35W-Brazil resolves to a loopback address: 127.0.1.1; using 192.168.18.83 instead (on interface wlp2s0)
23/09/20 11:26:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/20 11:26:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.read.json("../../datalake/twitter_datascience")

In [6]:
df.show()

+--------------------+--------------------+------------------+------------+
|                data|            includes|              meta|extract_date|
+--------------------+--------------------+------------------+------------+
|[{43, 19, 2023-08...|{[{2023-08-25T14:...|{1234567890abcdef}|  2023-08-25|
|[{2, 31, 2023-08-...|{[{2023-08-25T21:...|{1234567890abcdef}|  2023-08-25|
|[{97, 99, 2023-08...|{[{2023-08-25T16:...|{1234567890abcdef}|  2023-08-25|
|[{55, 38, 2023-08...|{[{2023-08-25T15:...|{1234567890abcdef}|  2023-08-25|
|[{74, 59, 2023-08...|{[{2023-08-25T14:...|              null|  2023-08-25|
|[{68, 32, 2023-09...|{[{2023-09-18T20:...|{1234567890abcdef}|  2023-09-19|
|[{62, 6, 2023-09-...|{[{2023-09-18T15:...|{1234567890abcdef}|  2023-09-19|
|[{78, 62, 2023-09...|{[{2023-09-18T02:...|{1234567890abcdef}|  2023-09-19|
|[{90, 32, 2023-09...|{[{2023-09-18T09:...|{1234567890abcdef}|  2023-09-19|
|[{61, 25, 2023-09...|{[{2023-09-18T22:...|              null|  2023-09-19|
|[{44, 49, 2

In [7]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [8]:
from pyspark.sql import functions as f

In [9]:
df.select(f.explode("data")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [10]:
df.select(f.explode("data")).show()

+--------------------+
|                 col|
+--------------------+
|{43, 19, 2023-08-...|
|{5, 27, 2023-08-2...|
|{92, 5, 2023-08-2...|
|{74, 21, 2023-08-...|
|{38, 36, 2023-08-...|
|{97, 36, 2023-08-...|
|{57, 33, 2023-08-...|
|{42, 13, 2023-08-...|
|{28, 50, 2023-08-...|
|{69, 22, 2023-08-...|
|{2, 31, 2023-08-2...|
|{60, 64, 2023-08-...|
|{37, 34, 2023-08-...|
|{17, 34, 2023-08-...|
|{71, 73, 2023-08-...|
|{44, 98, 2023-08-...|
|{32, 18, 2023-08-...|
|{77, 13, 2023-08-...|
|{14, 59, 2023-08-...|
|{7, 25, 2023-08-2...|
+--------------------+
only showing top 20 rows



In [11]:
tweet_df = df.select(f.explode("data").alias("tweets")).select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text")

In [12]:
tweet_df.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [13]:
tweet_df.show(5)

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|       43|             19|2023-08-25T03:48:...|  1|        49|         85|         58|           55|Tweet fictício ge...|
|        5|             27|2023-08-25T14:55:...| 27|        49|         46|         96|           79|Tweet fictício cr...|
|       92|              5|2023-08-25T14:42:...| 13|        54|         10|        100|           55|Tweet fictício cr...|
|       74|             21|2023-08-25T13:42:...| 82|        29|         13|         58|           14|Tweet fictício cr...|
|       38|             36|2023-08-25T22:10:...| 11|        16|         71|         83|           67|Tweet fictício cr...|
+---------+-----

In [14]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [19]:
df.select(f.explode("includes.users").alias("users")).select("users.*").printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)



In [21]:
users_df = df.select(f.explode("includes.users").alias("users")).select("users.*")

In [22]:
users_df.show(5)

+--------------------+---+------+--------+
|          created_at| id|  name|username|
+--------------------+---+------+--------+
|2023-08-25T14:18:...| 49|User 1|   user1|
|2023-08-25T22:48:...| 69|User 2|   user2|
|2023-08-25T21:37:...| 82|User 3|   user3|
|2023-08-25T09:50:...| 41|User 4|   user4|
|2023-08-25T06:49:...| 52|User 5|   user5|
+--------------------+---+------+--------+
only showing top 5 rows

